<a href="https://colab.research.google.com/github/RafikaSaadouni/CNN-LSTM/blob/main/lstm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [66]:
data_train=pd.read_csv('/content/drive/MyDrive/data/nsl-kdd/KDDTrain+.txt' ,delimiter=',', header=None)
data_test=pd.read_csv('/content/drive/MyDrive/data/nsl-kdd/KDDTest+.txt' ,delimiter=',', header=None)
data_test1=pd.read_csv('/content/drive/MyDrive/data/nsl-kdd/KDDTest-21.txt' ,delimiter=',', header=None)

data_train

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [67]:
import pandas as pd
new_data_train= pd.concat([data_train, data_test1]).drop_duplicates(keep=False)
new_data_train.reset_index(drop=True, inplace = True)
    

In [68]:
print(new_data_train.shape)
print(data_test.shape)

(136873, 43)
(22544, 43)


In [69]:
new_data_train.isnull().values.any()

False

In [70]:
data_test.describe()

,0,4,5,6,7,8,9,10,11,12,...,32,33,34,35,36,37,38,39,40,42
count,22544.000000,2.254400e+04,2.254400e+04,22544.000000,22544.000000,22544.000000,22544.000000,22544.000000,22544.000000,22544.000000,...,22544.000000,22544.000000,22544.000000,22544.000000,22544.000000,22544.000000,22544.000000,22544.000000,22544.000000,22544.000000
mean,218.859076,1.039545e+04,2.056019e+03,0.000311,0.008428,0.000710,0.105394,0.021647,0.442202,0.119899,...,140.750532,0.608722,0.090540,0.132261,0.019638,0.097814,0.099426,0.233385,0.226683,18.017965
std,1407.176612,4.727864e+05,2.121930e+04,0.017619,0.142599,0.036473,0.928428,0.150328,0.496659,7.269597,...,111.783972,0.435688,0.220717,0.306268,0.085394,0.273139,0.281866,0.387229,0.400875,4.270361
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,15.000000,0.070000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000
50%,0.000000,5.400000e+01,4.600000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,168.000000,0.920000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000
75%,0.000000,2.870000e+02,6.010000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,255.000000,1.000000,0.060000,0.030000,0.010000,0.000000,0.000000,0.360000,0.170000,21.000000
max,57715.000000,6.282565e+07,1.345927e+06,1.000000,3.000000,3.000000,101.000000,4.000000,1.000000,796.000000,...,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,21.000000


In [71]:
#les noms des colonne de dataset
featuresV=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]
#la colonne flag qui contient des valeur string
flagV=['OTH','RSTOS0','SF','SH','RSTO','S2','S1','REJ','S3','RSTR','S0']

#la colonne protocol_type qui contient des valeur string
protocol_typeV=['tcp','udp','icmp']
#la colonne service qui contient des valeur string
serviceV=['http','smtp','finger','domain_u','auth','telnet','ftp','eco_i','ntp_u','ecr_i','other','private','pop_3','ftp_data',
                  'rje','time','mtp','link','remote_job','gopher','ssh','name','whois','domain','login','imap4','daytime','ctf','nntp',
                  'shell','IRC','nnsp','http_443','exec','printer','efs','courier','uucp','klogin','kshell','echo','discard','systat',
                  'supdup','iso_tsap','hostnames','csnet_ns','pop_2','sunrpc','uucp_path','netbios_ns','netbios_ssn','netbios_dgm',
                  'sql_net','vmnet','bgp','Z39_50','ldap','netstat','urh_i','X11','urp_i','pm_dump','tftp_u','tim_i','red_i','icmp',
                  'http_2784','harvest','aol','http_8001']
# les étiquète 
binary_attack=['normal','ipsweep', 'nmap', 'portsweep','satan', 'saint', 'mscan','back', 'land', 'neptune', 'pod', 'smurf',
               'teardrop', 'apache2', 'udpstorm', 'processtable','mailbomb','buffer_overflow', 'loadmodule', 'perl', 'rootkit',
               'xterm', 'ps', 'sqlattack','ftp_write', 'guess_passwd', 'imap', 'multihop','phf', 'spy', 'warezclient',
               'warezmaster','snmpgetattack','named', 'xlock', 'xsnoop','sendmail', 'httptunnel', 'worm', 'snmpguess']

multiclass_attack={ 'normal': 'normal',
        'probe': ['ipsweep.', 'nmap.', 'portsweep.','satan.', 'saint.', 'mscan.'],
        'dos': ['back.', 'land.', 'neptune.', 'pod.', 'smurf.','teardrop.', 'apache2.', 'udpstorm.', 'processtable.','mailbomb.'],
        'u2r': ['buffer_overflow.', 'loadmodule.', 'perl.', 'rootkit.','xterm.', 'ps.', 'sqlattack.'],
        'r2l': ['ftp_write.', 'guess_passwd.', 'imap.', 'multihop.','phf.', 'spy.', 'warezclient.', 'warezmaster.','snmpgetattack.',
                   'named.', 'xlock.', 'xsnoop.','sendmail.', 'httptunnel.', 'worm.', 'snmpguess.']}  

In [72]:
new_data_train.columns=data_test.columns=featuresV


In [73]:

data_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal,21
22540,0,tcp,http,SF,317,938,0,0,0,0,...,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal,21
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back,15
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal,21


In [74]:
print(new_data_train.shape)
print(data_test.shape)

(136873, 43)
(22544, 43)


In [75]:
df_train_obj = new_data_train.iloc[:, :-2].select_dtypes(include='object')
df_train_num = new_data_train.iloc[:, :-2].select_dtypes(exclude='object')

print(f"shape of numeric features: {df_train_num.shape}")
print(f"shape of object features: {df_train_obj.shape}")

shape of numeric features: (136873, 38)
shape of object features: (136873, 3)


In [76]:
new_data_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136868,5,tcp,pop_3,SF,28,93,0,0,0,0,...,1.00,0.00,0.04,0.08,0.00,0.00,0.00,0.00,guess_passwd,6
136869,0,tcp,http,SF,336,285,0,0,0,0,...,0.92,0.02,0.00,0.00,0.00,0.00,0.05,0.00,normal,18
136870,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.37,0.03,0.01,0.02,0.05,0.08,0.85,0.58,mscan,13
136871,0,tcp,sunrpc,REJ,0,0,0,0,0,0,...,0.19,0.03,0.01,0.04,0.00,0.00,0.88,1.00,mscan,15


In [77]:
df_test_obj = data_test.iloc[:, :-2].select_dtypes(include='object')
df_test_num = data_test.iloc[:, :-2].select_dtypes(exclude='object')

print(f"shape of numeric features: {df_test_num.shape}")
print(f"shape of object features: {df_test_obj.shape}")

shape of numeric features: (22544, 38)
shape of object features: (22544, 3)


In [78]:
df_test_num.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,0,0,0,0,0,0,0,0,0,...,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00
1,0,0,0,0,0,0,0,0,0,0,...,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00
2,2,12983,0,0,0,0,0,0,0,0,...,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00
3,0,20,0,0,0,0,0,0,0,0,...,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00
4,1,0,15,0,0,0,0,0,0,0,...,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71


In [79]:
new_data_train['label']=new_data_train['label'].replace(['normal.','normal'],0)
for i in range(len(binary_attack)):
  new_data_train['label'] = new_data_train['label'].replace(binary_attack[i], 1)
y_train=new_data_train['label']
data_test['label']=data_test['label'].replace(['normal.','normal'],0)
for i in range(len(binary_attack)):
  data_test['label'] = data_test['label'].replace(binary_attack[i], 1)
y_test=data_test['label']
#supprimer la colonne la bel de dataset car on a créer une copier déja ===>séparation
del new_data_train['label']
del data_test['label']

In [80]:
y_train.shape

(136873,)

In [81]:
import numpy as np 
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

#encoder juste les  column qui contient string
data_train_enc = enc.fit_transform(df_train_obj).toarray()
train_enc_features = enc.get_feature_names_out(input_features=df_train_obj.columns)
data_test_enc = enc.transform(df_test_obj).toarray()
test_enc_features = enc.get_feature_names_out(input_features=df_test_obj.columns)

#concatiner string avec num columns
X_train_enc = np.c_[df_train_num, data_train_enc]
X_test_enc = np.c_[df_test_num, data_test_enc]
print(f"X_train_enc shape: {X_train_enc.shape}")
print(f"X_test_enc shape: {X_test_enc.shape}")
######################################
#encoder les labels
y_train=pd.get_dummies(y_train)
y_test=pd.get_dummies(y_test)
y_train.head()


X_train_enc shape: (136873, 122)
X_test_enc shape: (22544, 122)


,0,1
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [82]:
scaler = MinMaxScaler()
X_train_scaler = scaler.fit_transform(X_train_enc)
X_test_scaler = scaler.fit_transform(X_test_enc)
########################
y_train=scaler.fit_transform(y_train)
y_test=scaler.fit_transform(y_test)
y_train[0]

array([1., 0.])

In [83]:
x_train = np.reshape(X_train_scaler, (X_train_scaler.shape[0],1, X_train_scaler.shape[1]))
x_train.shape

(136873, 1, 122)

In [84]:
from sklearn.model_selection import train_test_split
# The next step is to split training and testing data. For this we will use sklearn function train_test_split().

x_train, x_test1 ,y_train, y_test1 = train_test_split(x_train, y_train, test_size=10000)
x_train.shape, x_test1.shape ,y_train.shape, y_test1.shape

((126873, 1, 122), (10000, 1, 122), (126873, 2), (10000, 2))

In [85]:
x_test = np.reshape(X_test_scaler, (X_test_scaler.shape[0],1,X_test_scaler.shape[1]))

x_test.shape

(22544, 1, 122)

In [86]:
X_train_enc.shape

(136873, 122)

In [87]:
X_train_extract = X_train_scaler.copy()
X_test_extract = X_test_scaler.copy()

In [88]:
from tensorflow import keras
look_back = X_train_extract.shape[1]
def multiClassModel(units=4, look_back=look_back):
    model = Sequential()
   # model.add(Input(shape=(1, look_back)))
    #model.add(LSTM(units=50, activation='relu'))
    model.add(LSTM(256, activation='relu', return_sequences=True, input_shape=(1, look_back)))
    model.add(Dropout(0.2))
    model.add(LSTM(128,activation='relu', return_sequences=True, input_shape = (1, look_back)))
    model.add(Dropout(0.2))
    
    model.add(LSTM(64,activation='relu', input_shape = (1, look_back)))
 
    #model.add(LSTM(100, activation='relu'))
   
    model.add(Dropout(0.2))
    
    model.add(Dense(units=32, activation='relu'))
    
    model.add(Dense(2, activation="sigmoid"))
    #opt = keras.optimizers.Adam(learning_rate= 0.0001)
    model.compile(loss="binary_crossentropy", optimizer='Adam' , metrics=['accuracy',keras.metrics.Precision()])
    model.summary()
    return model

In [ ]:
model = multiClassModel()
 
#X_train_lstm = X_train_extract.reshape(X_train_extract.shape[0], 1, X_train_extract.shape[1])
#X_test_lstm = X_test_extract.reshape(X_test_extract.shape[0], 1, X_test_extract.shape[1])
history=model.fit(x_train, y_train, epochs=80, batch_size=256, validation_data=(x_test,y_test))


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_19 (LSTM)              (None, 1, 256)            388096    
                                                                 
 dropout_21 (Dropout)        (None, 1, 256)            0         
                                                                 
 lstm_20 (LSTM)              (None, 1, 128)            197120    
                                                                 
 dropout_22 (Dropout)        (None, 1, 128)            0         
                                                                 
 lstm_21 (LSTM)              (None, 64)                49408     
                                                                 
 dropout_23 (Dropout)        (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 32)               

In [ ]:
tf.keras.utils.plot_model(model, 'my_first_model.png', show_shapes=True)

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

loss_val = history.history['val_loss']
acc_val = history.history['loss']
epochs = range(1,81)
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.plot(epochs, acc_val, 'g', label='train loss')
plt.title('Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc_val = history.history['val_accuracy']
acc_train = history.history['accuracy']
epochs = range(1,81)
plt.plot(epochs, acc_val, 'b', label='validation acc')
plt.plot(epochs, acc_train, 'g', label='train_accuracy')
plt.title('accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#check accuaracy of prediction
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score, recall_score
#model.score(x_valid,y_valid)

In [ ]:
#cf_matrix=confusion_matrix(y_valid, y_pred)
y_test_arg=np.argmax(y_test1,axis=1)
Y_pred = np.argmax(model.predict(x_test1),axis=1)
print('Confusion Matrix')
cf_matrix=confusion_matrix(y_test_arg, Y_pred)
print(cf_matrix)

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix, cmap='coolwarm',annot=True, linewidth=1,fmt="d")
plt.show()

In [ ]:
accuracy =accuracy_score(y_test_arg, Y_pred)*100
print(accuracy)

In [ ]:
recall = recall_score(y_test_arg, Y_pred , average="binary")
precision = precision_score(y_test_arg, Y_pred, average="binary")
FPR = cf_matrix[0][1]/(cf_matrix[0][1]+cf_matrix[1][1]) 

In [ ]:
print("Precision : " , precision*100)
print("Recall : ", recall*100)
print("Accuracy : ",accuracy)
print("FPR : ",FPR*100)